# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
pd.set_option('display.max_columns', 20)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

####################################################################################################
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_messages', con=engine)
X = df['message']
Y = df.iloc[:, 4:]
df.drop('child_alone', axis=1, inplace=True)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detect_url = re.findall(url_regex, text)
    for d_u in detect_url:
        text = text.replace(d_u, url_place_holder_string)
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(t).lower().strip() for t in tokens]
    return clean_tokens

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [7]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def test_model(y_test, y_pred):
    for index, column in enumerate(y_test):
        print(f'Column {index+1}: {column}')
        print(classification_report(y_test[column], y_pred[:, index]))
    accuracy = (y_pred == y_test.values).mean()
    print(f'Accuracy: {round(accuracy, 3)}')

In [9]:
y_pred = pipeline.predict(X_test)
test_model(y_test, y_pred)

Column 1: related
             precision    recall  f1-score   support

          0       0.64      0.36      0.46      1514
          1       0.83      0.94      0.88      5040

avg / total       0.79      0.81      0.78      6554

Column 2: request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5424
          1       0.76      0.54      0.63      1130

avg / total       0.88      0.89      0.88      6554

Column 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Column 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.81      3809
          1       0.77      0.60      0.67      2745

avg / total       0.76      0.76      0.75      6554

Column 5: medical_help
             precision    recall  f1-score 

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipeline__vect__tokenizer', 

In [11]:
parameters = {'clf__estimator__n_estimators':[10, 20], 
              'clf__estimator__learning_rate': [0.5, 1.0]}

In [12]:
cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, n_jobs=-1, verbose=3)
cv

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__learning_rate': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [13]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__features', 'estimator__clf', 'estimator__features__n_jobs', 'estimator__features__transformer_list', 'estimator__features__transformer_weights', 'estimator__features__text_pipeline', 'estimator__features__text_pipeline__memory', 'estimator__features__text_pipeline__steps', 'estimator__features__text_pipeline__vect', 'estimator__features__text_pipeline__tfidf', 'estimator__features__text_pipeline__vect__analyzer', 'estimator__features__text_pipeline__vect__binary', 'estimator__features__text_pipeline__vect__decode_error', 'estimator__features__text_pipeline__vect__dtype', 'estimator__features__text_pipeline__vect__encoding', 'estimator__features__text_pipeline__vect__input', 'estimator__features__text_pipeline__vect__lowercase', 'estimator__features__text_pipeline__vect__max_df', 'estimator__features__text_pipeline__vect__max_features', 'estimator__features__text_pipeline__vect__min_df', 'estimator__fea

In [14]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.2112890922959573, total=  34.0s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   43.0s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.22863682604272634, total=  33.9s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.1991353001017294, total=  33.8s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.21515768056968465, total=  33.6s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.21414038657171922, total=  34.0s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.22984998728705822, total=  57.7s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.245676500508647, total=  57.6s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_esti

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 18.4min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__learning_rate': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [15]:
cv.best_params_

{'clf__estimator__learning_rate': 1.0, 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)

In [17]:
test_model(y_test, y_pred)

Column 1: related
             precision    recall  f1-score   support

          0       0.66      0.29      0.41      1514
          1       0.82      0.95      0.88      5040

avg / total       0.78      0.80      0.77      6554

Column 2: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5424
          1       0.76      0.51      0.61      1130

avg / total       0.88      0.89      0.88      6554

Column 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Column 4: aid_related
             precision    recall  f1-score   support

          0       0.73      0.86      0.79      3809
          1       0.74      0.56      0.64      2745

avg / total       0.74      0.74      0.73      6554

Column 5: medical_help
             precision    recall  f1-score 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [18]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.